**HW-3**

In [1]:
import pyreadr
import torch
from torch.utils.data import Dataset , DataLoader
from sklearn.model_selection import train_test_split
import torch.nn as nn
import numpy as np
import torch.nn.functional as F

Construct a data set for this hw

In [2]:
class HW3_Data_set(Dataset):

    def __init__(self,dataset,dataset_labels):
        super().__init__()
        self.set = torch.from_numpy(dataset)
        self.set_labels = torch.from_numpy(dataset_labels)

    def __len__(self):
        return len(self.set_labels)

    def __getitem__(self, item):
        return self.set[item] , self.set_labels[item]

Logistic regression model

In [3]:
class LogisticRegression(nn.Module):

    def __init__(self, input_dim, output_dim):

        super().__init__()
        self.linear = nn.Linear(input_dim, output_dim)

    def forward(self, x):
        out = self.linear(x)
        return out

SVM RBF kernel

In [4]:
def rbf(dataset, vector,sigma):
    """
    :param dataset: [n,d]
    :param vector: [b,d]
    :param sigma : rbf sigma, must be positive
    :return: [b,n]
    """
    b = vector.shape[0]
    d = vector.shape[1]
    vectors = torch.chunk(vector,dim=0,chunks=b)
    batchDis = []
    for oneVec in vectors:
        # print("dataset",dataset)
        # print("x",oneVec)
        # print(dataset - oneVec.view(1,d))
        dis = torch.sum(torch.pow(dataset - oneVec.view(1,d), 2.), dim=1, keepdim=False)
        batchDis.append(dis)
    #print("before div sigma",-torch.stack(batchDis,dim=0))
    #print("After div sigma", -torch.stack(batchDis,dim=0) / (2. * sigma))
    #print("Final result " , torch.exp(-torch.stack(batchDis,dim=0) / (2. * sigma)))
    return torch.exp(-torch.stack(batchDis,dim=0) / (2. * sigma))

SVM Model

In [5]:
class SVM(nn.Module):

    def __init__(self,data_set, data_labels,sigma):
        """
        :param data_set: [n,d]
        :param data_labels: 1,-1 set [n]
        """
        super().__init__()
        self.data_set = torch.from_numpy(data_set).float()
        self.data_labels = torch.from_numpy(data_labels).float()
        self.n = self.data_set.size()[0]
        self.lambdas = nn.Parameter(torch.ones(size=[self.n],requires_grad=True).float(),requires_grad=True)
        self.b = nn.Parameter(torch.ones(1,requires_grad=True).float(),requires_grad=True)
        self.sigma2 = np.square(sigma)

    def forward(self,x):
        ## return [b]
        lambdas = F.relu(self.lambdas)
        result = self.data_labels.view(1,self.n) * lambdas.view(1,self.n) * rbf(self.data_set,x,self.sigma2)
        #print("Lambda ",lambdas)
        #print("Before sign ",torch.sum(result,dim=1) + self.b)
        return torch.tanh(torch.sum(result,dim=1) + self.b)


Neural Net Model

In [6]:
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu1 = nn.ReLU()
        self.fc3 = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        out = self.fc1(x)
        out = self.relu1(out)
        out = self.fc3(out)
        return out

Training function for Logistic regression and Neural net 

In [7]:
def train(model,lossCri,dataLoader,testDataLoader,epoch,weight_decay,lr,device = "cuda" if torch.cuda.is_available() else "cpu"):
    model = model.to(device)
    optimizer = torch.optim.SGD(model.parameters(),lr=lr,weight_decay=weight_decay,nesterov=True,momentum=0.9)
    lossCri = lossCri.to(device)
    for e in range(epoch):
        finalLoss = 0
        for times, (trainRows, trainLabels) in enumerate(dataLoader):
            trainRows = trainRows.float().to(device)
            trainLabels = trainLabels.long().to(device)
            optimizer.zero_grad()
            trainOutput = model(trainRows)
            loss = lossCri(trainOutput,trainLabels)
            loss.backward()
            optimizer.step()
            finalLoss = loss
        model = model.eval()
        # Calculate Accuracy
        correct = 0
        total = 0
        # Iterate through test dataset
        for testRows, testLabels in testDataLoader:
            testLabels = testLabels.to(device)
            # Forward pass only to get logits/output
            testOutputs = model(testRows.float().to(device))

            # Get predictions from the maximum value
            _, predicted = testOutputs.max(1)

            # Total number of labels
            total += testLabels.size(0)

            # Total correct predictions
            correct += predicted.eq(testLabels).sum().item()

        accuracy = 100 * correct / total

        # Print Loss
        print('Epoch: {}. Loss : {}.  Accuracy In Test Data set: {}'.format(e,finalLoss, accuracy))
        model = model.train(True)
    return model

Training function for SVM

In [8]:
def train_svm(model,dataLoader,testDataLoader,epoch,lr,weight_decay,device = "cuda" if torch.cuda.is_available() else "cpu"):
    model = model.to(device)
    optimizer = torch.optim.SGD(model.parameters(),lr=lr,weight_decay=weight_decay,nesterov=True,momentum=0.9)
    lossCri = nn.MSELoss().to(device)
    for e in range(epoch):
        for times, (trainRows, trainLabels) in enumerate(dataLoader):
            #print(times)
            trainRows = trainRows.float().to(device)
            trainLabels = trainLabels.float().to(device)
            optimizer.zero_grad()
            trainOutput = model(trainRows)
            loss = lossCri(trainOutput,trainLabels)
            # print("##########")
            # print(trainOutput)
            # print(trainLabels)
            # print(loss)
            loss.backward()
            optimizer.step()
        model = model.eval()
        # Calculate Accuracy
        correct = 0
        total = 0
        # Iterate through test dataset
        for testRows, testLabels in testDataLoader:
            testLabels = testLabels.to(device)
            # Forward pass only to get logits/output
            testOutputs = model(testRows.float().to(device))

            # Get predictions from the maximum value
            b = testOutputs.size(0)
            predicted = torch.where(testOutputs >= 0.,torch.ones(size=[b]),torch.zeros(size=[b])-1)
            total += testLabels.size(0)

            # Total correct predictions
            correct += predicted.eq(testLabels).sum().item()

        accuracy = 100 * correct / total

        # Print Result
        print('Epoch: {}.  Accuracy In Test Data set: {}%'.format(e, accuracy))
        model = model.train(True)
    return model


Load and processing data

In [9]:
data = pyreadr.read_r("./DeepFeature.RData")


deep_features = np.transpose(np.array(data["deep.feature"]).reshape([-1,2000]))
image_array = np.transpose(np.array(data["image.array"]).reshape([-1,2000]))
labelsChar = np.array(data["label"]).reshape([2000])
labels01 = np.array(labelsChar == "cat",dtype=np.float32)
labels1_1 = np.where(labelsChar == "cat",-1,1)


dp_train, dp_test, dp_label_train, dp_label_test = \
        train_test_split(deep_features, labels01, test_size=0.30)

ia_train, ia_test, ia_label_train, ia_label_test = \
        train_test_split(image_array, labels01, test_size=0.30)

dp_D = dp_train.shape[1]
ia_D = ia_train.shape[1]

Construct data set and data Loader For Logistic regression and Neural net

In [10]:
## train dataset
dp_train_set = HW3_Data_set(dataset=dp_train,dataset_labels=dp_label_train)
ia_train_set = HW3_Data_set(dataset=ia_train / 255.,dataset_labels=ia_label_train)

## test dataset
dp_test_set = HW3_Data_set(dataset=dp_test,dataset_labels=dp_label_test)
ia_test_set = HW3_Data_set(dataset=ia_test / 255.,dataset_labels=ia_label_test)

## data loader
dp_train_loader = DataLoader(dp_train_set,batch_size=25,shuffle=True)
ia_train_loader = DataLoader(ia_train_set,batch_size=25,shuffle=True)

dp_test_loader = DataLoader(dp_test_set,batch_size=25,shuffle=True)
ia_test_loader = DataLoader(ia_test_set,batch_size=25,shuffle=True)

logistic regression deep.feature

In [11]:
print("Logistic regression for deep.feature")
logisticR = LogisticRegression(dp_D,2)
lossCriL = nn.CrossEntropyLoss()
train(logisticR,lossCriL,dp_train_loader,dp_test_loader,epoch=15,weight_decay=5e-4,lr= 0.0001)

Logistic regression for deep.feature
Epoch: 0. Loss : 0.618233323097229.  Accuracy In Test Data set: 69.66666666666667
Epoch: 1. Loss : 0.5216636061668396.  Accuracy In Test Data set: 73.5
Epoch: 2. Loss : 0.43095821142196655.  Accuracy In Test Data set: 72.5
Epoch: 3. Loss : 0.40423375368118286.  Accuracy In Test Data set: 73.83333333333333
Epoch: 4. Loss : 0.4658602178096771.  Accuracy In Test Data set: 74.16666666666667
Epoch: 5. Loss : 0.30143672227859497.  Accuracy In Test Data set: 74.16666666666667
Epoch: 6. Loss : 0.42810672521591187.  Accuracy In Test Data set: 75.0
Epoch: 7. Loss : 0.42620915174484253.  Accuracy In Test Data set: 73.83333333333333
Epoch: 8. Loss : 0.38976600766181946.  Accuracy In Test Data set: 75.66666666666667
Epoch: 9. Loss : 0.32068726420402527.  Accuracy In Test Data set: 76.33333333333333
Epoch: 10. Loss : 0.5471965670585632.  Accuracy In Test Data set: 76.5
Epoch: 11. Loss : 0.40021923184394836.  Accuracy In Test Data set: 76.16666666666667
Epoch: 12.

LogisticRegression(
  (linear): Linear(in_features=4096, out_features=2, bias=True)
)

logistic regression image.array

In [12]:
print("Logistric regression for image.array")
logisticR = LogisticRegression(ia_D,2)
lossCriL = nn.CrossEntropyLoss()
train(logisticR,lossCriL,ia_train_loader,ia_test_loader,epoch=15,weight_decay=5e-4,lr= 0.0001)


Logistric regression for image.array
Epoch: 0. Loss : 0.6404321193695068.  Accuracy In Test Data set: 54.0
Epoch: 1. Loss : 0.7539743781089783.  Accuracy In Test Data set: 53.5
Epoch: 2. Loss : 0.6664098501205444.  Accuracy In Test Data set: 53.833333333333336
Epoch: 3. Loss : 0.6730889678001404.  Accuracy In Test Data set: 54.5
Epoch: 4. Loss : 0.7043527364730835.  Accuracy In Test Data set: 56.0
Epoch: 5. Loss : 0.6156076192855835.  Accuracy In Test Data set: 55.333333333333336
Epoch: 6. Loss : 0.6986448764801025.  Accuracy In Test Data set: 56.5
Epoch: 7. Loss : 0.6669578552246094.  Accuracy In Test Data set: 57.166666666666664
Epoch: 8. Loss : 0.6634848117828369.  Accuracy In Test Data set: 57.5
Epoch: 9. Loss : 0.6242170929908752.  Accuracy In Test Data set: 57.333333333333336
Epoch: 10. Loss : 0.6415241360664368.  Accuracy In Test Data set: 57.333333333333336
Epoch: 11. Loss : 0.7325840592384338.  Accuracy In Test Data set: 53.0
Epoch: 12. Loss : 0.5978148579597473.  Accuracy In 

LogisticRegression(
  (linear): Linear(in_features=3072, out_features=2, bias=True)
)

svm data loader construct

In [13]:
### svm data loader construct
dp_train_svm, dp_test_svm, dp_label_train_svm, dp_label_test_svm = \
        train_test_split(deep_features, labels1_1, test_size=0.30)

ia_train_svm, ia_test_svm, ia_label_train_svm, ia_label_test_svm = \
        train_test_split(image_array, labels1_1, test_size=0.30)

## train dataset
dp_train_set_svm = HW3_Data_set(dataset=dp_train_svm,dataset_labels=dp_label_train_svm)
ia_train_set_svm = HW3_Data_set(dataset=ia_train_svm / 255.,dataset_labels=ia_label_train_svm)

## test dataset
dp_test_set_svm = HW3_Data_set(dataset=dp_test_svm,dataset_labels=dp_label_test_svm)
ia_test_set_svm = HW3_Data_set(dataset=ia_test_svm / 255.,dataset_labels=ia_label_test_svm)

## data loader
dp_train_loader_svm = DataLoader(dp_train_set_svm,batch_size=1,shuffle=True)
ia_train_loader_svm = DataLoader(ia_train_set_svm,batch_size=1,shuffle=True)

dp_test_loader_svm = DataLoader(dp_test_set_svm,batch_size=1,shuffle=False)
ia_test_loader_svm = DataLoader(ia_test_set_svm,batch_size=1,shuffle=False)



SVM for deep.feature

In [14]:
print("SVM for deep.feature.")
svm = SVM(dp_train_svm,dp_label_train_svm,sigma=36)
train_svm(svm,dp_train_loader_svm,dp_test_loader_svm,epoch=25,lr = 0.0001,weight_decay=5e-4)

SVM for deep.feature.
Epoch: 0.  Accuracy In Test Data set: 74.5%
Epoch: 1.  Accuracy In Test Data set: 75.5%
Epoch: 2.  Accuracy In Test Data set: 75.16666666666667%
Epoch: 3.  Accuracy In Test Data set: 75.0%
Epoch: 4.  Accuracy In Test Data set: 75.33333333333333%
Epoch: 5.  Accuracy In Test Data set: 75.16666666666667%
Epoch: 6.  Accuracy In Test Data set: 75.5%
Epoch: 7.  Accuracy In Test Data set: 76.0%
Epoch: 8.  Accuracy In Test Data set: 75.16666666666667%
Epoch: 9.  Accuracy In Test Data set: 74.33333333333333%
Epoch: 10.  Accuracy In Test Data set: 75.83333333333333%
Epoch: 11.  Accuracy In Test Data set: 75.83333333333333%
Epoch: 12.  Accuracy In Test Data set: 75.66666666666667%
Epoch: 13.  Accuracy In Test Data set: 74.5%
Epoch: 14.  Accuracy In Test Data set: 75.16666666666667%
Epoch: 15.  Accuracy In Test Data set: 74.5%
Epoch: 16.  Accuracy In Test Data set: 74.83333333333333%
Epoch: 17.  Accuracy In Test Data set: 74.83333333333333%
Epoch: 18.  Accuracy In Test Data s

SVM()

SVM for image.array

In [15]:
print("SVM for image.array.")
svm = SVM(ia_train_svm / 255.,ia_label_train_svm,sigma=18)
train_svm(svm,ia_train_loader_svm,ia_test_loader_svm,epoch=25,lr = 0.0001,weight_decay=0)

SVM for image.array.
Epoch: 0.  Accuracy In Test Data set: 55.833333333333336%
Epoch: 1.  Accuracy In Test Data set: 58.666666666666664%
Epoch: 2.  Accuracy In Test Data set: 58.833333333333336%
Epoch: 3.  Accuracy In Test Data set: 58.666666666666664%
Epoch: 4.  Accuracy In Test Data set: 56.166666666666664%
Epoch: 5.  Accuracy In Test Data set: 59.0%
Epoch: 6.  Accuracy In Test Data set: 58.833333333333336%
Epoch: 7.  Accuracy In Test Data set: 59.0%
Epoch: 8.  Accuracy In Test Data set: 59.166666666666664%
Epoch: 9.  Accuracy In Test Data set: 59.333333333333336%
Epoch: 10.  Accuracy In Test Data set: 59.5%
Epoch: 11.  Accuracy In Test Data set: 58.666666666666664%
Epoch: 12.  Accuracy In Test Data set: 57.0%
Epoch: 13.  Accuracy In Test Data set: 58.666666666666664%
Epoch: 14.  Accuracy In Test Data set: 59.166666666666664%
Epoch: 15.  Accuracy In Test Data set: 59.333333333333336%
Epoch: 16.  Accuracy In Test Data set: 58.666666666666664%
Epoch: 17.  Accuracy In Test Data set: 57.

SVM()

Neural net for deep.feature

In [16]:
print("Neural net for deep.feature")
NNt = NeuralNet(dp_D,hidden_size=2000,num_classes=2)
lossCriL = nn.CrossEntropyLoss()
train(NNt,lossCriL,dp_train_loader,dp_test_loader,epoch=25,weight_decay=5e-4,lr= 0.0001)


Neural net for deep.feature
Epoch: 0. Loss : 0.6050152778625488.  Accuracy In Test Data set: 64.83333333333333
Epoch: 1. Loss : 0.5596890449523926.  Accuracy In Test Data set: 70.0
Epoch: 2. Loss : 0.5768333077430725.  Accuracy In Test Data set: 71.33333333333333
Epoch: 3. Loss : 0.4804919958114624.  Accuracy In Test Data set: 72.66666666666667
Epoch: 4. Loss : 0.5203630924224854.  Accuracy In Test Data set: 74.0
Epoch: 5. Loss : 0.47613945603370667.  Accuracy In Test Data set: 74.16666666666667
Epoch: 6. Loss : 0.4962328374385834.  Accuracy In Test Data set: 74.16666666666667
Epoch: 7. Loss : 0.37645038962364197.  Accuracy In Test Data set: 75.16666666666667
Epoch: 8. Loss : 0.627378523349762.  Accuracy In Test Data set: 75.0
Epoch: 9. Loss : 0.5210436582565308.  Accuracy In Test Data set: 75.16666666666667
Epoch: 10. Loss : 0.30259668827056885.  Accuracy In Test Data set: 75.33333333333333
Epoch: 11. Loss : 0.48448115587234497.  Accuracy In Test Data set: 75.33333333333333
Epoch: 12.

NeuralNet(
  (fc1): Linear(in_features=4096, out_features=2000, bias=True)
  (relu1): ReLU()
  (fc3): Linear(in_features=2000, out_features=2, bias=True)
)

Neural net for image.array

In [17]:
print("Neural net for image.array")
NNt = NeuralNet(ia_D,hidden_size=2000,num_classes=2)
lossCriL = nn.CrossEntropyLoss()
train(NNt,lossCriL,ia_train_loader,ia_test_loader,epoch=25,weight_decay=5e-4,lr= 0.0001)


Neural net for image.array
Epoch: 0. Loss : 0.6889950633049011.  Accuracy In Test Data set: 51.0
Epoch: 1. Loss : 0.6793661713600159.  Accuracy In Test Data set: 53.5
Epoch: 2. Loss : 0.6936144828796387.  Accuracy In Test Data set: 54.833333333333336
Epoch: 3. Loss : 0.7094132304191589.  Accuracy In Test Data set: 54.666666666666664
Epoch: 4. Loss : 0.678005576133728.  Accuracy In Test Data set: 55.666666666666664
Epoch: 5. Loss : 0.6816192865371704.  Accuracy In Test Data set: 55.166666666666664
Epoch: 6. Loss : 0.653522789478302.  Accuracy In Test Data set: 55.0
Epoch: 7. Loss : 0.6258469223976135.  Accuracy In Test Data set: 55.166666666666664
Epoch: 8. Loss : 0.7234441637992859.  Accuracy In Test Data set: 55.333333333333336
Epoch: 9. Loss : 0.674009382724762.  Accuracy In Test Data set: 56.0
Epoch: 10. Loss : 0.6038647890090942.  Accuracy In Test Data set: 56.166666666666664
Epoch: 11. Loss : 0.6934240460395813.  Accuracy In Test Data set: 55.333333333333336
Epoch: 12. Loss : 0.67

NeuralNet(
  (fc1): Linear(in_features=3072, out_features=2000, bias=True)
  (relu1): ReLU()
  (fc3): Linear(in_features=2000, out_features=2, bias=True)
)